In [25]:
import cv2
import os
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
from tqdm import tqdm

# Suppress warnings
warnings.filterwarnings('ignore')

sns.set_style("whitegrid")
sns.set_palette('rainbow')
sns.set_context('talk')

basedir = '/home/jack/Mounts/DiskOne/kona_coffee/imagemasks'

os.makedirs(basedir, exist_ok=True)

df = pd.read_csv('classified.csv')

# drop unknown and overripe lifecycles
df = df[df.lifecycle != 'unknown']
df = df[df.lifecycle != 'overripe']

# filter if mask or image doesn't exist on disk
# df = df[df['image'].apply(lambda x: os.path.exists(x))]
# df = df[df['mask'].apply(lambda x: os.path.exists(x))]

df

,image,x,y,mask,lifecycle
0,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,299,58,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,ripening
4,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,109,178,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,ripe
5,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,114,125,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,ripening
6,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,175,169,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,unripe
7,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,190,238,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,unripe
...,...,...,...,...,...
2393,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,192,160,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,ripening
2394,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,331,55,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,unripe
2395,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,368,18,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,unripe
2396,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,331,102,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,unripe


In [26]:
def get_display_image(row):
    cv2image = cv2.imread(row.image)

    mask = np.load(row.mask)
    mask = mask.astype(np.uint8)
    
    # apply the mask to the image
    mask_image = cv2.bitwise_and(cv2image, cv2image, mask=mask)
    
    # crop the image to the mask
    mask_image = mask_image[np.ix_(mask.any(1),mask.any(0))]
    
    return mask_image


fdf = []

for row in tqdm(df.itertuples(), total=len(df)):
    image_path = os.path.join(basedir, f'{row.mask.split("/")[-1]}.png')

    mask_image = get_display_image(row)
    
    if mask_image is not None:
        cv2.imwrite(image_path, mask_image)
    else:
        print('None image for ', image_path)
        
    fdf.append((image_path, row.lifecycle))
    
df = pd.DataFrame(fdf, columns=['image', 'lifecycle'])

df

100%|██████████| 2287/2287 [00:10<00:00, 226.03it/s]


,image,lifecycle
0,/home/jack/Mounts/DiskOne/kona_coffee/imagemas...,ripening
1,/home/jack/Mounts/DiskOne/kona_coffee/imagemas...,ripe
2,/home/jack/Mounts/DiskOne/kona_coffee/imagemas...,ripening
3,/home/jack/Mounts/DiskOne/kona_coffee/imagemas...,unripe
4,/home/jack/Mounts/DiskOne/kona_coffee/imagemas...,unripe
...,...,...
2282,/home/jack/Mounts/DiskOne/kona_coffee/imagemas...,ripening
2283,/home/jack/Mounts/DiskOne/kona_coffee/imagemas...,unripe
2284,/home/jack/Mounts/DiskOne/kona_coffee/imagemas...,unripe
2285,/home/jack/Mounts/DiskOne/kona_coffee/imagemas...,unripe


In [27]:
df.to_csv('classified_masks.csv', index=False)

In [28]:
df

,image,lifecycle
0,/home/jack/Mounts/DiskOne/kona_coffee/imagemas...,ripening
1,/home/jack/Mounts/DiskOne/kona_coffee/imagemas...,ripe
2,/home/jack/Mounts/DiskOne/kona_coffee/imagemas...,ripening
3,/home/jack/Mounts/DiskOne/kona_coffee/imagemas...,unripe
4,/home/jack/Mounts/DiskOne/kona_coffee/imagemas...,unripe
...,...,...
2282,/home/jack/Mounts/DiskOne/kona_coffee/imagemas...,ripening
2283,/home/jack/Mounts/DiskOne/kona_coffee/imagemas...,unripe
2284,/home/jack/Mounts/DiskOne/kona_coffee/imagemas...,unripe
2285,/home/jack/Mounts/DiskOne/kona_coffee/imagemas...,unripe
